# Web Analysis with GeoDjango

In this chapter, we will cover the following topics:
<ul>
    <li>Setting up a GeoDjango web application</li>
    <li>Creating an indoor web routing service</li>
    <li>Visualizing an indoor routing service</li>
    <li>Creating an indoor route-type service</li>
    <li>Creating an indoor route from room to room</li>
 </ul>

## Introduction

Our final chapter is all about extending our analysis into a web application using the <strong>Django</strong> web framework. One of the standard Django contributed packages is known as <strong>GeoDjango</strong> and is found in the <em>django/contrib/gis</em> package. This is a feature-packed GIS toolset for geospatial web application development. The spatial libraries used here depend on the spatial database backend that you choose. For PostgreSQL the library requirements include GEOS, PROJ.4, and PostGIS.

Django is known for its good documentation and the <em>gis contrib</em> package installation is no exception, having its own set of instructions for you to follow at https://docs.djangoproject.com/en/dev/ref/contrib/gis/.

Since GeoDjango is part of the standard Django installation, you will see that your first step is to install the Django framework. For any reference on installing GeoDjango, PostgreSQL, and PostGIS, take a look at Chapter 1, Setting Up Your Geospatial Python Environment.

## Setting up a GeoDjango Web Application

We need to get some basic Django groundwork done and this will be a very high-level fly over at setting up the required basics to start a Django web application. Check out the official Django tutorials for further information at https://docs.djangoproject.com/en/dev/intro/tutorial01/.

### Getting ready

We are going to build a routing web service using the Django REST framework (http://www.django-rest-framework.org/). All that we need to implement is a basic web service that you can install with the help of <em>pip</em>:

<code> $ pip install djangorestframework </code>

### How to do it...

Let's now create a Django project using the django-admin tool as follows:

1. From the command line, enter the <em>/ch11/code</em> directory and execute this command:

<code>$ django-admin startproject web_analysis</code>

2. Now you will have a <em>/ch11/code/web_analysis/web_analysis<em> directory and inside it, you'll find all the standard basic Django components.

3. To create our web service, we are going to place all the services into a Django App called <em>api</em>. This app will store all our services. Creating this <em>api</em> application is as easy as typing this code:

<code> $  cd web_analysis </code>

Change into the newly created <em>web_analysis</em> directory:

<code> $ django-admin startapp api </code>

Now create your new application called "api".
    
4. This creates a new <em>/ch11/code/web_analysis/api</em> folder and inside it you will find the default installed Django app files. Next, we need to tell Django about the Django REST Framework, GeoDjango gis app, and our new api application; we do this in our <em>/ch11/code/web_analysis/web_analysis/settings.py</em> file. Let's add the lines <em>'django.contrib.gis', 'rest_framework'</em>, and <em>'api'</em> to our <em>INSTALLED_APPS</em> variable as follows:

<code>
    INSTALLED_APPS = (
        'django.contrib.admin',
        'django.contrib.auth',
        'django.contrib.contenttypes',
        'django.contrib.sessions',
        'django.contrib.messages',
        'django.contrib.staticfiles',

        #### GeoDjango Contrib APP
        # 'django.contrib.gis',

        #### third party apps
        'rest_framework',
        
        ##### our local apps
        'api', 
        
    )
</code>

5. To enable the GeoDjango spatial models and spatial capabilities, <em>'django.contrib.gis'</em> will allow us to access the rich geospatial framework. We have it commented out at this point since we are not going to use it until later, but feel free to uncomment it as this will do no harm. This spatial framework requires a spatial database and we will use PostgreSQL with PostGIS as our backend. Let's go ahead and change the database connection now in our <em>settings.py</em> as follows:

<code>
    DATABASES = {
        'default': {
            # PostgreSQL with PostGIS
            'ENGINE': 'django.contrib.gis.db.backends.postgis',
            'NAME': 'py_geoan_cb', # DB name
            'USER': 'saturn', # DB user name
            'PASSWORD': 'secret', # DB user password
            'HOST': 'localhost',
            'PORT': '5432',
        }
    }
</code>

<pre><strong> Note</strong>

    The database here is referencing the same PostgreSQL + PostGIS database that we created earlier on in Chapter 3, Moving Spatial Data from One Format to Another. Visit the Converting a Shapefile to a PostGIS table using ogr2ogr, recipe in Chapter 3, Moving Spatial Data from One Format to Another where we created the <em>spatial</em> database, if you are going to skip ahead to this section.</pre>
    
6. Our final <em>settings.py</em> configuration is set up to log errors and exceptions to a log file, catching errors if any occur. First up, we'll create a new folder called <em>/web_analysis/logs</em> and add two new files called <em>debug.log</em> and <em>verbose.log</em>. We will write any errors that occur into these two files and log a request or simply print out an error to these files. So, go ahead and copy this code into the bottom of your <em>/web_analysis/web_analysis/settings.py</em> file as follows:

<code>
    LOGGING_CONFIG = None

    LOGGING = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'verbose': {
                'format' : "[%(asctime)s] %(levelname)s [%(name)s:%(lineno)s] %(message)s",
                'datefmt' : "%d/%b/%Y %H:%M:%S"
            },
            'simple': {
                'format': '%(levelname)s %(message)s'
            },
        },
        'handlers': {
            'file_verbose': {
                'level': 'DEBUG',
                'class': 'logging.FileHandler',
                'filename': 'logs/verbose.log',
                'formatter': 'verbose'
            },
            'file_debug': {
                'level': 'DEBUG',
                'class': 'logging.FileHandler',
                'filename': 'logs/debug.log',
                'formatter': 'verbose'
            },
        },
        'loggers': {
            'django': {
                'handlers':['file_verbose'],
                'propagate': True,
                'level':'DEBUG',
            },
            'api': {
                'handlers': ['file_debug'],
                'propagate': True,
                'level': 'DEBUG',
            },

        }
    }

    import logging.config
    logging.config.dictConfig(LOGGING)
</code>

7. Next up, let's create a new database user and a separate PostgreSQL schema to store all our Django-related tables; otherwise, all the new Django tables will automatically be created in the PostgreSQL default schema public. Our new user is called <em>saturn</em> and can log in with the <em>secret</em> password. To create a new user, you can use the command-line tool that's run as the <em>postgres</em> user:

<code> createuser saturn </code>

    You can also use the PGAdmin free tool. On Ubuntu, don't forget to change to the <em>postgres</em> user that will allow you to create a new user on your database.
    
8. Now, let's create a new schema called <em>django</em> that will store all our Django application tables. Use PGAdmin or the SQL command to do this as follows:

    <code> CREATE SCHEMA django AUTHORIZATION saturn; </code>

9. With this new schema in place, we only need to assign the PostgreSQL <em>search_path</em> variable order to set the <em>django</em> schema as the first priority. To accomplish this, we need to use the SQL <em>ALTER ROLE</em> command as follows:

<code>ALTER ROLE saturn SET search_path = django, geodata, public, topology; </code>

10. This sets the <em>search_path</em> order defining <em>django</em> as the first schema, <em>geodata</em> as the second, and so forth. This order is for all database connections for the <em>saturn</em> user. When we create our new Django tables, all of them will now automatically be created inside the <em>django</em> schema.
    
11. Let's go ahead now and initialize our Django project and create all the tables as follows:

<code> python manage.py migrate </code>

12. The built-in Django <em>manage.py</em> command calls the <em>migrate</em> function and performs the sync in one go. Next, let's create a superuser for our application who can login and have full control of the entire web application. Then, follow the command-line instructions to enter the username, e-mail, and password as follows:

<code> python manage.py createsuperuser </code>

13. With all these steps now completed, we are ready to actually get something done and build our online routing application. To test whether everything is working, run this command:

    <code> python manage.py runserver 8000 </code>

14. Open up your local web browser and see the welcome Django default page.

## Creating an indoor web routing service

Let's take all the effort we put into Chapter 8, Network Routing Analysis, out onto the World Wide Web. Our routing service will simply accept a starting point location, an x, y coordinate pair, a floor level, and a destination location. The indoor routing service will then calculate the shortest path and return a complete route in the form of a GeoJSON file.

### Getting ready

To layout the tasks ahead, let's list out what we need to accomplish at a high level so that we're clear about where we are going:

1. Create a URL pattern to call a route service.
2. Build a view to handle an incoming URL request and deliver the appropriate GeoJSON route web response:
        1. Accept incoming request parameters.

        Start x coordinate.

        Start y coordinate.

        Start floor number.

        End x coordinate.

        End y coordinate.

        End floor number.
        2. Return GeoJSON LineString.

        Route geometry.

        Route length.

        Route walk time.

We also need to let our new database user named saturn in order to have access to the tables located in the PostgreSQL geodata schema created in Chapter 8, Network Routing Analysis. Currently, only the user named postgres is the owner and almighty one. This needs to change so that we can keep on trucking without needing to recreate our tables as created in Chapter 8, Network Routing Analysis. So, let's go ahead and simply make the saturn user the owner of each of these tables as follows:

<code>
    ALTER TABLE geodata.ch08_e01_networklines OWNER TO saturn;
    ALTER TABLE geodata.ch08_e01_networklines_vertices_pgr OWNER TO saturn;
    ALTER TABLE geodata.ch08_e02_networklines OWNER TO saturn;
    ALTER TABLE geodata.ch08_e02_networklines_vertices_pgr  OWNER TO saturn;
    ALTER TABLE geodata.networklines_3857 OWNER TO saturn;
    ALTER TABLE geodata.networklines_3857_vertices_pgr OWNER TO saturn;
    
</code>

<strong>Tip</strong>
If you are looking for a way to allow both the saturn user and any other user to gain access to these tables, you could create a PostgreSQL group role and assign the user to this role as follows:

<code>
    CREATE ROLE gis_edit   VALID UNTIL 'infinity';
    GRANT ALL ON SCHEMA geodata TO GROUP gis_edit;
    GRANT gis_edit TO saturn;
    GRANT ALL ON TABLE geodata.ch08_e01_networklines TO GROUP gis_edit;
    GRANT ALL ON TABLE geodata.ch08_e01_networklines_vertices_pgr TO GROUP gis_edit;
    GRANT ALL ON TABLE geodata.ch08_e02_networklines TO GROUP gis_edit;
    GRANT ALL ON TABLE geodata.ch08_e02_networklines_vertices_pgr TO GROUP gis_edit;
    GRANT ALL ON TABLE geodata.networklines_3857 TO GROUP gis_edit;
    GRANT ALL ON TABLE geodata.networklines_3857_vertices_pgr TO GROUP gis_edit;
</code>

### How to do it...
Our code is now in one folder in a structure that's common to all Django web projects, so following these steps should be straightforward:

1. Let's begin by wiring up our new URL. Go ahead and open up the urls.py file inside your ch11/code/web_analysis/ folder. Inside the file, you will need to enter the main URL configuration for our new web page. This file was automatically created when we created the project. Django fills in some helper text, as you can see, that shows you some basic configuration options. We need to add the admin app, which we will use later, and the URL for our new API. The API application will have its very own URL configuration file as you can see in the api.urls references, which we will create next. The /web_analysis/urls.py file should look like this:

<pre><code>
    """web_analysis URL Configuration

    The `urlpatterns` list routes URLs to views. For more information please see:
        https://docs.djangoproject.com/en/1.8/topics/http/urls/
    Examples:
    Function views
        1. Add an import:  from my_app import views
        2. Add a URL to urlpatterns:  url(r'^$', views.home, name='home')
    Class-based views
        1. Add an import:  from other_app.views import Home
        2. Add a URL to urlpatterns:  url(r'^$', Home.as_view(), name='home')
    Including another URLconf
        1. Add an import:  from blog import urls as blog_urls
        2. Add a URL to urlpatterns:  url(r'^blog/', include(blog_urls))
    """
    from django.conf.urls import include, url
    from django.contrib import admin

    urlpatterns = [
        url(r'^admin/', include(admin.site.urls)),
        url(r'^api/', include('api.urls')),
    ]
</code></pre>

2. Next up, let's create the /web_analysis/api/urls.py api URLs. This file is not automatically generated so we'll create this file now. The content of this /api/urls.py file will be as follows:

<pre><code>
    from django.conf.urls import patterns, url
    from rest_framework.urlpatterns import format_suffix_patterns

    urlpatterns = patterns('api.views',
        #  ex valid call from to /api/directions/1587848.414,5879564.080,2&1588005.547,5879736.039,2
        url(r'^directions/(?P<start_coord>[-]?\d+\.?\d+,\d+\.\d+),(?P<start_floor>\d+)&(?P<end_coord>[-]?\d+\.?\d+,\d+\.\d+),(?P<end_floor>\d+)/$', 'create_route', name='directions'),

    )

    urlpatterns = format_suffix_patterns(urlpatterns)
</code></pre>

3. The regular expression looks wild as most regular expressions do. If you need some help understanding it, try referring to https://regex101.com/#python. Go ahead and paste this regular expression in the regular expression field:

<code>
    (?P<start_coord>[-]?\d+\.?\d+,\d+\.\d+),(?P<start_floor>\d+)&(?P<end_coord>[-]?\d+\.?\d+,\d+\.\d+),(?P<end_floor>\d+)
</code>

4. To test your URL string, simply paste this text in the TEST STRING field:

<code>1587848.414,5879564.080,2&1588005.547,5879736.039,2</code>

5. If it's lit up in some funky colors, you are good to go:
<img src="./50790OS_11_01.jpg" height=200 width=400>

Django's use of regular expressions for URL configuration is quite handy but not always obvious and explicit to read. Our URL is explained in a textual manner and would read like this:
<code>/api/directions/start_x,start_y,start_floor&end_x,end_y,end_floor</code>

This is a real example from your development machine. When calling the URL, it will look like this:

<code>http://localhost:8000/api/directions/1587848.414,5879564.080,2&1588005.547,5879736.039,2</code>

The start and end location information is separated with an & symbol, while the contents of each start parameter and end parameter are separated by a comma.

Going forward, in terms of complexity, we now need to enter the logic part of our API. Django handles this in the views. Our /web_analysis/api/views.py code contains the code to handle the request and response.

6. The main def create_route function should look familiar as it is taken directly from Chapter 8, Network Routing Analysis, with some modifications. A new helper function is created called find_closest_network_node. This new function is more robust and faster than our previous SQL that we used to find the node closest to any given x, y coordinate entered by a user:

<pre><code>
    #!/usr/bin/env python
    # -*- coding: utf-8 -*-

    import traceback
    from django.http import HttpResponseNotFound
    from rest_framework.decorators import api_view
    from rest_framework.response import Response
    from geojson import loads, Feature, FeatureCollection
    import logging
    logger = logging.getLogger(__name__)
    from django.db import connection


    def find_closest_network_node(x_coord, y_coord, floor):
        """
        Enter a given coordinate x,y and floor number and
        find the nearest network node
        to start or end the route on
        :param x_coord: float  in epsg 3857
        :param y_coord: float  in epsg 3857
        :param floor: integer value equivalent to floor such as
    	2  = 2nd floor
        :return: node id as an integer
        """
        # connect to our Database
        logger.debug("now running function find_closest_network_node")
        cur = connection.cursor()

        # find nearest node on network within 200 m
        # and snap to nearest node
        query = """ SELECT
            verts.id as id
            FROM geodata.networklines_3857_vertices_pgr AS verts
            INNER JOIN
              (select ST_PointFromText('POINT(%s %s %s)', 3857)as geom) AS pt
            ON ST_DWithin(verts.the_geom, pt.geom, 200.0)
            ORDER BY ST_3DDistance(verts.the_geom, pt.geom)
            LIMIT 1;"""

        # pass 3 variables to our %s %s %s place holder in query
        cur.execute(query, (x_coord, y_coord, floor,))

        # get the result
        query_result = cur.fetchone()

        # check if result is not empty
        if query_result is not None:
            # get first result in tuple response there is only one
            point_on_networkline = int(query_result[0])
            return point_on_networkline
        else:
            logger.debug("query is none check tolerance value of 200")
            return False


    # use the rest_framework decorator to create our api
    #  view for get, post requests
    @api_view(['GET', 'POST'])
    def create_route(request, start_coord, start_floor, end_coord, end_floor):
        """
        Generate a GeoJSON indoor route passing in a start x,y,floor
        followed by &  then the end x,y,floor
        Sample request: http:/localhost:8000/api/directions/1587848.414,5879564.080,2&1588005.547,5879736.039,2
        :param request:
        :param start_coord: start location x,y
        :param start_floor: floor number  ex)  2
        :param end_coord: end location x,y
        :param end_floor: end floor ex)  2
        :return: GeoJSON route
        """

        if request.method == 'GET' or request.method == 'POST':

            cur = connection.cursor()

            # parse the incoming coordinates and floor using
            # split by comma
            x_start_coord = float(start_coord.split(',')[0])
            y_start_coord = float(start_coord.split(',')[1])
            start_floor_num = int(start_floor)

            x_end_coord = float(end_coord.split(',')[0])
            y_end_coord = float(end_coord.split(',')[1])
            end_floor_num = int(end_floor)

            # use our helper function to get vertices
            # node id for start and end nodes
            start_node_id = find_closest_network_node(x_start_coord,
                               y_start_coord,
                               start_floor_num)

            end_node_id = find_closest_network_node(x_end_coord,
                               y_end_coord,
                               end_floor_num)

            routing_query = '''
                SELECT seq, id1 AS node, id2 AS edge,
                  total_cost AS cost, layer,
                  type_id, ST_AsGeoJSON(wkb_geometry) AS geoj
                  FROM pgr_dijkstra(
                    'SELECT ogc_fid as id, source, target,
                         st_length(wkb_geometry) AS cost,
                         layer, type_id
                     FROM geodata.networklines_3857',
                    %s, %s, FALSE, FALSE
                  ) AS dij_route
                  JOIN  geodata.networklines_3857 AS input_network
                  ON dij_route.id2 = input_network.ogc_fid ;
              '''

            # run our shortest path query
            if start_node_id or end_node_id:
                cur.execute(routing_query, (start_node_id, end_node_id))
            else:
                logger.error("start or end node is None "
                             + str(start_node_id))
                return HttpResponseNotFound('<h1>Sorry NO start or end  node'
                                ' found within 200m</h1>')

            # get entire query results to work with
            route_segments = cur.fetchall()

            # empty list to hold each segment for our GeoJSON output
            route_result = []

            # loop over each segment in the result route segments
            # create the list of our new GeoJSON
            for segment in route_segments:
                seg_cost = segment[3]      # cost value
                layer_level = segment[4]   # floor number
                seg_type = segment[5]
                geojs = segment[6]         # geojson coordinates
                geojs_geom = loads(geojs)  # load string to geom
                geojs_feat = Feature(geometry=geojs_geom,
                                     properties={'floor': layer_level,
                                                  'length': seg_cost,
                                                  'type_id': seg_type})
                route_result.append(geojs_feat)

            # using the geojson module to create our GeoJSON Feature Collection
            geojs_fc = FeatureCollection(route_result)

            try:
                return Response(geojs_fc)
            except:
                logger.error("error exporting to json model: "+ str(geojs_fc))
                logger.error(traceback.format_exc())
                return Response({'error': 'either no JSON or no key params in your JSON'})
        else:
            retun HttpResponseNotFound('<h1>Sorry not a GET or POST request</h1>')

</code></pre>

The resulting API call has a nice web interface that's automatically generated by the Django REST Framework as shown in the following screenshot. The URL you need to call is also shown and should return a GeoJSON result.

<img src="./50790OS_11_02.jpg" height=300 width=300>

The following URL will return GeoJSON to your browser; in Chrome, it will normally just show up as simple text. IE users may download it as a file by simply opening it in Notepad++ or a local text editor to see the contents of GeoJSON:
<code>http://localhost:8000/api/directions/1587848.414,5879564.080,2&1588005.547,5879736.039,2/?format=json </code>

### How it works...
Our view handles the request and response using the Django REST Framework. There are two functions that do all the hard work without ever using the Django Object Relational Mapper (ORM). The reason for this is two-fold: first, to show you the basics of direct Database usage without too much abstraction and the inner workings of what is going on; second, because we are using functions of PostGIS that are not available directly through the ORM of GeoDjango, such as ST_3DDistance or ST_PointFromText. We could use some of the fancy Django helpers, such as .extra(), but this would confuse everyone but an experienced Django user.

Let's discuss the first find_closest_network_node function that takes three parameters: x_coord, y_coord, and floor. The x and y coordinates should be double precision float values, while the floor is an integer. Our regular expression URL limits any request to digits so there is no need to do any extra format checking in our code.

The SQL query that finds the nearest node and returns its ID limits the search radius to 200 m, which would equal one huge room or auditorium. Then, we order by the 3D distance between the points and LIMIT the result to one since we are not routing to multiple locations.

This feeds our second function called create_route where we pass it the start coordinate, start floor integer, end coordinate, and end floor number. Our URL at /web_analysis/api/urls.py uses a regular expression named groups that corresponds to the same names used in the request parameters of our function. This keeps things more explicit so that you know what values belong where in a query.

We begin with parsing the incoming parameters to get the exact values as floats and integers to feed our routing query. The routing query itself is unchanged from Chapter 8, Network Routing Analysis, so refer to this chapter for more details. The Django REST framework response sends the GeoJSON back to the client and has the ability to return it as raw text as well.

## Visualizing an indoor routing service

With our wonderful API created, it's time now to visualize this indoor route returned as GeoJSON on a map. We will now dive into the Django template components to create the HTML, JS, and CSS for our front-facing web page that displays a simple slippy web map using Openlayers 3.4.0. and Bootstrap CSS.

Our new web map will display the GeoJSON on the map with a nice style alongside a menu bar where we will include later functionality.
<img src="./50790OS_11_03.jpg" height=400 width=400>

### Getting ready

We need to build a few new folders and files to store new static and template content for our Django web application. Let's begin doing this by creating the /web_analysis/templates folder followed by the /web_analysis/static folder.

Inside our /static/ folder, we will place the nondynamic content of the JavaScript and CSS files. The /templates/ folder will store the HTML template files used to create our web pages.

Next up, let's tell Django /web_analysis/settings.py about the location of our new templates folder; add the os.path.join(BASE_DIR, 'templates') value to the 'DIRS' key shown here so that the TEMPLATES variable looks like this:

<pre><code>
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [os.path.join(BASE_DIR,  'templates'),],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]
</code></pre>

To manage our maps, lets create a new Django application called maps where we can store all our map information as follows:

<code> > python manage.py startapp maps </code>

Next, register your new app in the /web_analysis/web_analysis/settings.py INSTALLED APPS variable by adding this under the api entry 'maps', under the entry 'api'.

The /maps/urls.py file is not automatically created so let's do this now and fill in some content as follows:

<code>
    from django.conf.urls import patterns, url
from rest_framework.urlpatterns import format_suffix_patterns

urlpatterns = patterns('maps.views',
    #  ex valid call from to /api/directions/1587848.414,5879564.080,2&1588005.547,5879736.039,2
    url(r'^(?P<map_name>\w+)/$', 'route_map', name='route-map'),

)

urlpatterns = format_suffix_patterns(urlpatterns)
</code>

We need to assign maps/urls.py within our main /web_analysis/web_analysis/urls.py so that we can freely create any URL for all our mapping needs.

Add this line to the /web_analysis/web_analysis/urls.py file as follows:

<code>url(r'^maps/', include('maps.urls')),</code>

This means that all the URL's inside our /maps/urls.py will start with http://localhost:8000/maps/.

We are now ready to set up the static files and static contents inside settings.py as follows:

<code>
STATIC_URL = '/static/'
STATIC_FOLDER = 'static'

STATICFILES_DIRS = [
   os.path.join(BASE_DIR, STATIC_FOLDER),
]

# finds all static folders in all apps
STATICFILES_FINDERS = (
    'django.contrib.staticfiles.finders.FileSystemFinder',
    'django.contrib.staticfiles.finders.AppDirectoriesFinder',
)
</code>

You should have the following folders and files now in the /static/ folder:

<code>
static
+---css
|       bootstrap-responsive.min.css
|       bootstrap.min.css
|       custom-layout.css
|       font-awesome.min.css
|       ol.css
|
+---img
\---js
        bootstrap.min.js
        jquery-1.11.2.min.js
        jquery.min.js
        ol340.js
</code>

This should be enough to set up your Django project in order for it to serve up a static map.

### How to do it...

Actually serving up the map requires us to create an HTML page. We use the built-in Django template engine to build two HTML pages. The first page template is base.html that will hold the basics of our web map page, making it a very important part of our frontend design. What's included in this page is a set of block tags, each for separate content place holders. This allows us to quickly create new map pages based on our base template, which sets up our basic template architecture.

1. Here is the /templates/base.html file:

<code>
    {% load staticfiles %}
    <<!DOCTYPE html>>
    <html lang="en">
    <head>
        {% block head %}

        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <meta name="description" content="Sample Map">
        <meta name="author" content="Michael Diener">
        <meta charset="UTF-8">

        <title>{% block title %}Default Title{% endblock %}</title>

        <script src="{% static "js/jquery-1.11.2.min.js" %}"></script>
        <link rel="stylesheet" href="{% static "css/bootstrap.min.css" %}">
        <script src="{% static "js/bootstrap.min.js" %}"></script>
        <link rel="stylesheet" href="{% static "css/ol.css" %}" type="text/css">
        <link rel="stylesheet" href="{% static "css/custom-layout.css" %}" type="text/css">
        <script src="{% static "js/ol340.js" %}"></script>

        {% endblock head %}
    </head>
    <body>
    {% block body %}

        {% block nav %}
            <nav class="navbar navbar-inverse navbar-fixed-top">
              <div class="container">
                <div class="navbar-header">
                  <button type="button" class="navbar-toggle collapsed" data-toggle="collapse" data-target="#navbar" aria-expanded="false" aria-controls="navbar">
                    <span class="sr-only">Toggle navigation</span>
                    <span class="icon-bar"></span>
                    <span class="icon-bar"></span>
                    <span class="icon-bar"></span>
                  </button>
                  <a class="navbar-brand" href="#">Indoor Project</a>
                </div>
                <div id="navbar" class="collapse navbar-collapse">
                  <ul class="nav navbar-nav">
                    <li><a href="#about">About</a></li>
                    <li><a href="#contact">Contact</a></li>
                  </ul>
                </div><!--/.nav-collapse -->
              </div>
            </nav>
        {% endblock nav %}


    {% endblock body %}
    </body>
    </html>
</code>

2. Now, let's move on to the actual map. A new template called /templates/route-map.html contains all the actual Django template blocks that are filled with HTML content as follows:

<code>
    {% extends "base.html" %}
    {% load staticfiles %}

    {% block title %}Simple route map{% endblock %}

    {% block body %}

    {{ block.super }}

    <div class="container-fluid">

        <div class="row">
          <div class="col-md-2">
            <div id="directions" class="directions">
                <form>
                    <div class="radio">
                      <label>
                        <input type="radio" name="typeRoute" id="routeTypeStandard" value="0" checked>
                        Standard Route
                      </label>
                    </div>
                    <div class="radio">
                      <label>
                        <input type="radio" name="typeRoute" id="routeTypeBarrierFree" value="1">
                        Barrier Free Route
                      </label>
                    </div>
                  <button type="submit" class="btn btn-default">Submit</button>
                    <br>
                </form>

            </div>
          </div>
          <div class="col-md-10">
            <div id="map" class="map"></div>
          </div>

        </div>
    </div>



        <script>

               var routeUrl = '/api/directions/1587848.414,5879564.080,2&1588005.547,5879736.039,2&' + sel_Val2  + '/?format=json';

              map.getLayers().push(new ol.layer.Vector({
                        source: new ol.source.GeoJSON({url: routeUrl, crossDomain: true,}),
                        style:  new ol.style.Style({
                            stroke: new ol.style.Stroke({
                              color: 'blue',
                              width: 4
                            })
                          }),
                        title: "Route",
                        name: "Route"
                    }));

            });
            var vectorLayer = new ol.layer.Vector({
                            source: new ol.source.GeoJSON({url: geojs_url}),
                            style:  new ol.style.Style({
                                stroke: new ol.style.Stroke({
                                  color: 'red',
                                  width: 4
                                })
                              }),
                            title: "Route",
                            name: "Route"
                        });

            var map = new ol.Map({
              layers: [
                new ol.layer.Tile({
                  source: new ol.source.OSM()
                }),
                vectorLayer
              ],
              target: 'map',
              controls: ol.control.defaults({
                attributionOptions: /** @type {olx.control.AttributionOptions} */ ({
                  collapsible: false
                })
              }),
              view: new ol.View({
                center: [1587927.09817072,5879650.90059265],
                zoom: 18
              })
            });

            </script>

    {% endblock body %}
</code>

3. For our application to actually show these templates, we need to create a view. The view handles the request and serves route-map.html in return. Now, our simple view is complete:

<code>
    from django.shortcuts import render

    def route_map(request):
        return render(request, 'route-map.html')
</code>

### How it works...

Starting with the base.html template, we set out the basic building blocks for map making. The static files and resources were set up to handle serving our JavaScript and CSS code. The base.html file is designed to allow us to add elements that are shared between multiple HTML pages such as a master page in Microsoft PowerPoint. The more blocks, that is, place holders, the better your base.

Our route-map.html contains the actual code referencing our api by calling it with a predefined, hardcoded from, to URL:

<code>
var geojs_url = "http://localhost:8000/api/directions/1587898.414,5879564.080,1&1588005.547,5879736.039,2/?format=json"
</code>

The /maps/views.py code is where any map logic, variables, or parameters are passed around to the template. In our code, we simply take in a request and return an HTML page. Now you have a rudimentary indoor routing service and visualization client to show off to your friends.

## Creating an indoor route-type service

Building a route based on a specified type value, such as a Barrier Free Route or Standard Pedestrian Route value, is great for your users. How to build different route types is based on the available data connected to our indoor graph of ways. This example will allow a user to select the barrier-free route and our service will generate a path, avoiding obstacles such as stairs:
<img src="./50790OS_11_04.jpg" height=400 width=400>

### Getting ready

We need to access some more data on our network to allow routing types. The type of route is based on a network line type, which is stored as an attribute on each LineString. To classify our route types, we have the following lookup table schema:
<table>
    <tr>
        <th>Value</th>
        <th>Route Type</th>
    </tr>
    <tr>
        <td>0</td>
        <td>Indoor route</td>
    </tr>
    <tr>
        <td>1</td>
        <td>Outdoor route</td>
    </tr>
    <tr>
        <td>2</td>
        <td>Elevator</td>
    </tr>
    <tr>
        <td>3</td>
        <td>Stairs</td>
    </tr>
</table>

Therefore, we want to avoid any stairs segments, which technically means avoiding <em>type_id = 3</em>.

<strong>Note</strong>
Optionally, you could create a lookup table to store all the possible types and their equivalent weights. These values could then be included in the calculation of the total cost value to influence the route outcome

Now we can control how the route is generated based on certain preferences as well. A standard route search can now be set for preferences, such as taking the stairs over the elevator or vice versa, depending on your needs:

<code>
ALTER TABLE geodata.ch08_e01_networklines ADD COLUMN total_cost double precision;
ALTER TABLE geodata.ch08_e02_networklines ADD COLUMN total_cost double precision;

update geodata.networklines_3857 set total_cost = st_length(wkb_geometry)*88
where type_id = 2;

update geodata.networklines_3857 set total_cost = st_length(wkb_geometry)*1.8
where type_id = 3;
</code>

If you update <em>geodata.networklines_3857</em>, make sure the user <em>saturn</em> is the owner or has access; otherwise, your API call will break.

### How to do it...

The least-cost path from any point is controlled by a basic property called cost. For a standard route, the cost is equal to the distance of a segment. We search for the least-cost path, which means finding the shortest path to our destination.

To control the path, we set the cost values. Creating a barrier-free route involves setting all segment types equal to stairs at an extraordinarily high value so that the path, that is, the distance, is huge and is, therefore, excluded in the shortest path route finding process. Our other option is to add a WHERE clause to the query and only accept values where type_id is not equal to 3, which means that it is not of the type stairs. We are going to use this option in our upcoming code.

Therefore our data needs to be clean in order to allow us to assign specific costs to specific segment types in our network lines.

Now, we need to add a new parameter to capture the route type:

1. We'll update the /api/views.py function, create route(), and add a new parameter called route_type. Next up is the actual query that needs to accept this new parameter. We set up a new variable called barrierfree_q to hold the WHERE clause that we will add to our original query:

<code>
def create_route(request, start_coord, start_floor, end_coord, end_floor, route_type):
        base_route_q = """SELECT ogc_fid as id, source, target,
                         total_cost AS cost,
                         layer, type_id
                         FROM geodata.networklines_3857"""

        # set default query
        barrierfree_q = "WHERE 1=1"
        if route_type == "1":
            # exclude all networklines of type stairs
            barrierfree_q = "WHERE type_id not in (3,4)"


        routing_query = '''
            SELECT seq, id1 AS node, id2 AS edge,
              ST_Length(wkb_geometry) AS cost, layer,
              type_id, ST_AsGeoJSON(wkb_geometry) AS geoj
              FROM pgr_dijkstra('
                {normal} {type}', %s, %s, FALSE, FALSE
              ) AS dij_route
              JOIN  geodata.networklines_3857 AS input_network
              ON dij_route.id2 = input_network.ogc_fid ;
          '''.format(normal=base_route_q, type=barrierfree_q)
     
</code>


2. We'll update our /api/urls.py to input our new URL parameter, route_type. The newly added named group regular expression is naturally called route_type and only accepts numbers from 0 to 9. This then, of course, also limits you to 10 route types. So, if you want to add more types, you will need to update your regex as follows:

<code>
from django.conf.urls import patterns, url
from rest_framework.urlpatterns import format_suffix_patterns

urlpatterns = patterns('api.views',
    #  ex valid call from to /api/directions/1587848.414,5879564.080,2&1588005.547,5879736.039,2
    url(r'^directions/(?P<start_coord>[-]?\d+\.?\d+,\d+\.\d+),(?P<start_floor>\d+)&(?P<end_coord>[-]?\d+\.?\d+,\d+\.\d+),(?P<end_floor>\d+)&(?P<route_type>[0-9])/$', 'create_route', name='directions'),

)

urlpatterns = format_suffix_patterns(urlpatterns)

</code>

3. The /maps/views.py function needs a facelift too so that we can pass in the parameters. Now, it will accept route_type as defined in our /api/urls.py:

<code>
from django.shortcuts import render


def route_map(request, route_type = "0"):

    return render(request, 'route-map.html', {'route_type': route_type})

</code>

4. It's time to update route-map.html to include radio buttons that allow a user to select either a Standard Route or Barrier Free Route. The map will then update the route as soon as you click on the route type radio button:

<code>
{% extends "base.html" %}
{% load staticfiles %}

{% block title %}Simple route map{% endblock %}

{% block body %}

{{ block.super }}

<div class="container-fluid">

    <div class="row">
      <div class="col-md-2">
        <div id="directions" class="directions">
            <form>
                <div class="radio">
                  <label>
                    <input type="radio" name="typeRoute" id="routeTypeStandard" value="0" checked>
                    Standard Route
                  </label>
                </div>
                <div class="radio">
                  <label>
                    <input type="radio" name="typeRoute" id="routeTypeBarrierFree" value="1">
                    Barrier Free Route
                  </label>
                </div>
              <button type="submit" class="btn btn-default">Submit</button>
                <br>
            </form>

        </div>
      </div>
      <div class="col-md-10">
        <div id="map" class="map"></div>
      </div>

    </div>
</div>

    <script>
        var url_base = "/api/directions/";
        var start_coord = "1587848.414,5879564.080,2";
        var end_coord =  "1588005.547,5879736.039,2";
        var r_type = {{ route_type }};
        var geojs_url = url_base + start_coord + "&" + end_coord + "&" + sel_Val + '/?format=json';
        var sel_Val = $( "input:radio[name=typeRoute]:checked" ).val();

        $( ".radio" ).change(function() {
           map.getLayers().pop();
           var sel_Val2 = $( "input:radio[name=typeRoute]:checked" ).val();
           var routeUrl = '/api/directions/1587848.414,5879564.080,2&1588005.547,5879736.039,2&' + sel_Val2  + '/?format=json';

          map.getLayers().push(new ol.layer.Vector({
                    source: new ol.source.GeoJSON({url: routeUrl, crossDomain: true,}),
                    style:  new ol.style.Style({
                        stroke: new ol.style.Stroke({
                          color: 'blue',
                          width: 4
                        })
                      }),
                    title: "Route",
                    name: "Route"
                }));

        });
        var vectorLayer = new ol.layer.Vector({
                        source: new ol.source.GeoJSON({url: geojs_url}),
                        style:  new ol.style.Style({
                            stroke: new ol.style.Stroke({
                              color: 'red',
                              width: 4
                            })
                          }),
                        title: "Route",
                        name: "Route"
                    });

        var map = new ol.Map({
          layers: [
            new ol.layer.Tile({
              source: new ol.source.OSM()
            }),
            vectorLayer
          ],
          target: 'map',
          controls: ol.control.defaults({
            attributionOptions: /** @type {olx.control.AttributionOptions} */ ({
              collapsible: false
            })
          }),
          view: new ol.View({
            center: [1587927.09817072,5879650.90059265],
            zoom: 18
          })
        });

        </script>

{% endblock body %}

</code>

Our results for type = 0 or a route using stairs should look like this:

<img src="./50790OS_11_05.jpg" height=400 width=400>

The barrier-free route will use type = 1, which means forced elevator use and avoiding all stairs. Your result should then look like this:

<img src="./50790OS_11_06.jpg" height=400 width=400>

### How it works...

The main part to understand here is that we need to add an option route type to our API call. This API call must accept a specific route type that we have defined as a number from 0 to 9. This route type number is then passed to our URL as a parameter and api/views.py runs the call. The API then generates a new route based on the route type.

All our changes are made inside the /api/view.py code that now includes a SQL WHERE clause and excludes networklines with a type_id = 3—that is, stairs. This query change keeps our app fast without actually increasing any Django middleware code in our views.

The frontend needs the user to select a route type with the default route type set to a standard value, such as 0, as in the case of stairs. This default type is used because in most indoor environments the stairs are usually shorter. You can, of course, change this default to whatever value or criteria you'd like at any time. A radio select box is used to restrain the choice to either a standard route or a barrier-free route. Upon selecting a route type, the map automatically removes the old route and creates a new route.

## Creating an indoor route from room to room

Routing from room A to room B in an indoor routing web application over multiple floors with routing types brings together all our work up to this point. We will import some room data and utilize our network to then allow a user to select a room, route from one room to the next, and select a type of route.

<img src="./50790OS_11_07.jpg" width=400 height=400>

### Getting ready

We need to import a set of room polygons for both the first and second floor as follows:

1. Import a Shapefile of the first floor room polygons as follows:

<code>ogr2ogr -a_srs EPSG:3857 -lco "SCHEMA=geodata" -lco "COLUMN_TYPES=name=varchar,room_num=integer,floor=integer" -nlt POLYGON -nln ch11_e01_roomdata -f PostgreSQL "PG:host=localhost port=5432 user=saturn dbname=py_geoan_cb password=secret" e01_room_data.shp </code>

2. Import a Shapefile of the second floor room polygons as follows:

<code>ogr2ogr -a_srs EPSG:3857 -lco "SCHEMA=geodata" -lco "COLUMN_TYPES=name=varchar,room_num=integer,floor=integer" -nlt POLYGON -nln ch11_e02_roomdata -f PostgreSQL "PG:host=localhost port=5432 user=saturn dbname=py_geoan_cb password=secret" e02_room_data.shp </code>

3. Create a new PostgreSQL view to merge all the new room data into one table so that we can query all the rooms at once:

<code>
    CREATE OR REPLACE VIEW  geodata.search_rooms_v AS
    SELECT floor, wkb_geometry, room_num FROM geodata.ch11_e01_roomdata 
    UNION
    SELECT floor, wkb_geometry, room_num FROM geodata.ch11_e02_roomdata ;
    ALTER TABLE geodata.search_rooms_v OWNER TO saturn;
</code>


### How to do it...

To allow a user to route from A to B, we need to enable a route from a field and a route to a field, as follows:

1. Create a new URL to accept the new parameter of the start and end room number. The first URL for example will look like http://localhost:8000/api/directions/10010&20043&0, which means that the route from room number 10010 to room number 20042 using the standard route type equals to zero.

<strong>NOTE</strong>
The second URL is an extra function that you can call to only return the center coordinate of a room when you pass in the room number like this: http://localhost:8000/directions/10010.

This function in the view does not exist and is left for you to do as homework.

2. Build a new /api/views.py function to find a room center coordinate and return the nearest node on networklines to this coordinate:

<code>
def get_room_centroid_node(room_number):
    '''
    Find the room center point coordinates
    and find the closest route node point
    :param room_number: integer value of room number
    :return: Closest route node to submitted room number
    '''

    room_center_q = """SELECT  floor,
            ST_asGeoJSON(st_centroid(wkb_geometry))
            AS geom FROM geodata.search_rooms_v
            WHERE room_num = %s;"""

    cur = connection.cursor()
    cur.execute(room_center_q, (room_number,))

    res = cur.fetchall()

    res2 = res[0]

    room_floor = res2[0]
    room_geom_x = json.loads(res2[1])
    room_geom_y = json.loads(res2[1])

    x_coord = float(room_geom_x['coordinates'][0])
    y_coord = float(room_geom_y['coordinates'][1])

    room_node = find_closest_network_node(x_coord, y_coord, room_floor)
    try:
        return room_node
    except:
        logger.error("error get room center " + str(room_node))
        logger.error(traceback.format_exc())
        return {'error': 'error get room center'}
</code>

3. Build the function inside /api/views.py to accept a start node ID, end node ID, and a route type that will then return a GeoJSON of the final route as follows:

<code>
def run_route(start_node_id, end_node_id, route_type):
    '''

    :param start_node_id:
    :param end_node_id:
    :param route_type:
    :return:
    '''

    cur = connection.cursor()
    base_route_q = """SELECT ogc_fid AS id, source, target,
                     total_cost AS cost,
                     layer, type_id
                     FROM geodata.networklines_3857"""

    # set default query
    barrierfree_q = "WHERE 1=1"
    if route_type == "1":
        # exclude all networklines of type stairs
        barrierfree_q = "WHERE type_id not in (3,4)"

    routing_query = '''
        SELECT seq, id1 AS node, id2 AS edge,
          ST_Length(wkb_geometry) AS cost, layer,
          type_id, ST_AsGeoJSON(wkb_geometry) AS geoj
          FROM pgr_dijkstra('
            {normal} {type}', %s, %s, FALSE, FALSE
          ) AS dij_route
          JOIN  geodata.networklines_3857 AS input_network
          ON dij_route.id2 = input_network.ogc_fid ;
      '''.format(normal=base_route_q, type=barrierfree_q)

    # run our shortest path query
    if start_node_id or end_node_id:
        cur.execute(routing_query, (start_node_id, end_node_id))
    else:
        logger.error("start or end node is None "
                     + str(start_node_id))
        return HttpResponseNotFound('<h1>Sorry NO start or end node'
                                    ' found within 200m</h1>')

    # get entire query results to work with
    route_segments = cur.fetchall()

    # empty list to hold each segment for our GeoJSON output
    route_result = []

    # loop over each segment in the result route segments
    # create the list of our new GeoJSON
    for segment in route_segments:
        seg_cost = segment[3]  # cost value
        layer_level = segment[4]  # floor number
        seg_type = segment[5]
        geojs = segment[6]  # geojson coordinates
        geojs_geom = loads(geojs)  # load string to geom
        geojs_feat = Feature(geometry=geojs_geom,
                             properties={'floor': layer_level,
                                         'length': seg_cost,
                                         'type_id': seg_type})
        route_result.append(geojs_feat)

    # using the geojson module to create our GeoJSON Feature Collection
    geojs_fc = FeatureCollection(route_result)

    return geojs_fc
</code>

4. At last, we can create a function that our API will call to generate a response:

<code>
@api_view(['GET', 'POST'])
def route_room_to_room(request, start_room_num, end_room_num, route_type):
    '''
    Generate a GeoJSON route from room number
    to room number
    :param request: GET or POST request
    :param start_room_num: an integer room number
    :param end_room_num: an integer room number
    :param route_type: an integer room type
    :return: a GeoJSON linestring of the route
    '''

    if request.method == 'GET' or request.method == 'POST':

        start_room = int(start_room_num)
        end_room = int(end_room_num)

        start_node_id = get_room_centroid_node(start_room)
        end_node_id = get_room_centroid_node(end_room)

        res = run_route(start_node_id, end_node_id, route_type)

        try:
            return Response(res)
        except:
            logger.error("error exporting to json model: " + str(res))
            logger.error(traceback.format_exc())
            return Response({'error': 'either no JSON or no key params in your JSON'})
    else:
        return HttpResponseNotFound('<h1>Sorry not a GET or POST request</h1>')
</code>

5. Add a URL to /api/urls.py to access a list of all available rooms:

<code>url(r'^rooms/$', 'room_list', name='room-list'),</code>

6. Create an API service to return a JSON array of all room numbers. This array is used in autocomplete fields, route-from, and route-to. We use the Twitter Typeahead.js JavaScript library to handle our autocomplete dropdown type hinting. As a user, all you need to do is type 1, for example, and all the rooms beginning with 1 will show up as 10010 (check this out at http://twitter.github.io/typeahead.js/examples/):

<code>
@api_view(['GET', 'POST'])
def room_list(request):
    '''
    http://localhost:8000/api/rooms
    :param request: no parameters GET or POST
    :return: JSON Array of room numbers
    '''
    cur = connection.cursor()
    if request.method == 'GET' or request.method == 'POST':

        room_query = """SELECT room_num FROM geodata.search_rooms_v"""

        cur.execute(room_query)
        room_nums = cur.fetchall()

        room_num_list = []
        for x in room_nums:
            v = x[0]
            room_num_list.append(v)

        try:
            return Response(room_num_list)
        except:
            logger.error("error exporting to json model: " + str(room_num_list))
            logger.error(traceback.format_exc())
            return Response({'error': 'either no JSON or no key params in your JSON'})
</code>

7. Our final base.html template is complete, containing all the spice needed for our final route from room to room as follows:

<code>
{% load staticfiles %}
<!DOCTYPE html>
<html lang="en">
<head>
    {% block head %}

    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta name="description" content="Sample Map">
    <meta name="author" content="Michael Diener">
    <meta charset="UTF-8">

    <title>{% block title %}Default Title{% endblock %}</title>

    <script src="{% static "js/jquery-1.11.2.min.js" %}"></script>
    <link rel="stylesheet" href="{% static "css/bootstrap.min.css" %}">
    <script src="{% static "js/bootstrap.min.js" %}"></script>
    <link rel="stylesheet" href="{% static "css/ol.css" %}" type="text/css">
    <link rel="stylesheet" href="{% static "css/custom-layout.css" %}" type="text/css">
    <script src="{% static "js/ol340.js" %}"></script>

    {% endblock head %}
</head>
<body>
{% block body %}

    {% block nav %}
        <nav class="navbar navbar-inverse navbar-fixed-top">
          <div class="container">
            <div class="navbar-header">
              <button type="button" class="navbar-toggle collapsed" data-toggle="collapse" data-target="#navbar" aria-expanded="false" aria-controls="navbar">
                <span class="sr-only">Toggle navigation</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
              </button>
              <a class="navbar-brand" href="http://www.indrz.com" target="_blank">Indoor Project</a>
            </div>
            <div id="navbar" class="collapse navbar-collapse">
              <ul class="nav navbar-nav">
                <li><a href="#about" target="_blank">About</a></li>
                <li><a href="https://github.com/mdiener21/" target="_blank">Contact</a></li>
              </ul>
            </div><!--/.nav-collapse -->
          </div>
        </nav>
    {% endblock nav %}


{% endblock body %}
</body>
</html>
</code>

8. Now we'll create our final route-map.html template and the JavaScript that goes with it as follows:

<code>
{% extends "base.html" %}
{% load staticfiles %}

{% block title %}Simple route map{% endblock %}

{% block head %}
{{ block.super }}
    <script src="{% static "js/bloodhound.min.js" %}"></script>
    <script src="{% static "js/typeahead.bundle.min.js" %}"></script>
{% endblock head %}

{% block body %}

{{ block.super }}

<div class="container-fluid">

    <div class="row">
      <div class="col-md-2">
        <div id="directions" class="directions">
            <form id="submitForm">
              <div id="rooms-prefetch" class="form-group">
                <label for="route-to">Route From:</label>
                <input type="text" class="typeahead form-control" id="route-to" placeholder="Enter Room Number">
              </div>
              <div id="rooms-prefetch" class="form-group">
                <label for="route-from">Route To:</label>
                <input type="text" class="typeahead form-control" id="route-from" placeholder="Enter Room Number">
              </div>

                <div class="radio">
                  <label>
                    <input type="radio" name="typeRoute" id="routeTypeStandard" value="0" checked>
                    Standard Route
                  </label>
                </div>
                <div class="radio">
                  <label>
                    <input type="radio" name="typeRoute" id="routeTypeBarrierFree" value="1">
                    Barrier Free Route
                  </label>
                </div>
              <button id="enterRoute" type="submit" class="btn btn-default">Go !</button>
                <br>
            </form>

        </div>
      </div>
      <div class="col-md-10">
        <div id="map" class="map"></div>
      </div>

    </div>
</div>

<script>  {% include 'routing.js' %} </script>


<script>
    var roomNums = new Bloodhound({
      datumTokenizer: Bloodhound.tokenizers.whitespace,
      queryTokenizer: Bloodhound.tokenizers.whitespace,
      prefetch: 'http://localhost:8000/api/rooms/?format=json'
    });

    // passing in `null` for the `options` arguments will result in the default
    // options being used
    $('#rooms-prefetch .typeahead').typeahead(null, {
      name: 'countries',
        limit: 100,
      source: roomNums
    });


    $( "#submitForm" ).submit(function( event ) {
    {#  alert( "Handler for .submit() called."  );#}
        var startNum = $('#route-from').val();
        var endNum = $('#route-to').val();
        var rType = $( "input:radio[name=typeRoute]:checked" ).val();
         addRoute(startNum, endNum, rType);
      event.preventDefault();
    });


</script>


{% endblock body %}
</code>

9. Our maps/templates/routing.js contains the functions needed to call the routing API as follows:

<code>
        var url_base = "/api/directions/";
        var start_coord = "1587848.414,5879564.080,2";
        var end_coord =  "1588005.547,5879736.039,2";
        var sel_Val = $( "input:radio[name=typeRoute]:checked" ).val();
        var geojs_url = url_base + start_coord + "&" + end_coord + "&" + sel_Val + '/?format=json';

// uncomment this code if you want to reactivate
// the quick static demo switcher
        //$( ".radio" ).change(function() {
        //   map.getLayers().pop();
        //   var sel_Val2 = $( "input:radio[name=typeRoute]:checked" ).val();
        //   var routeUrl = '/api/directions/1587848.414,5879564.080,2&1588005.547,5879736.039,2&' + sel_Val2  + '/?format=json';
        //
        //  map.getLayers().push(new ol.layer.Vector({
        //            source: new ol.source.GeoJSON({url: routeUrl}),
        //            style:  new ol.style.Style({
        //                stroke: new ol.style.Stroke({
        //                  color: 'blue',
        //                  width: 4
        //                })
        //              }),
        //            title: "Route",
        //            name: "Route"
        //        }));
        //
        //});

        var vectorLayer = new ol.layer.Vector({
                        source: new ol.source.GeoJSON({url: geojs_url}),
                        style:  new ol.style.Style({
                            stroke: new ol.style.Stroke({
                              color: 'red',
                              width: 4
                            })
                          }),
                        title: "Route",
                        name: "Route"
                    });

        var map = new ol.Map({
          layers: [
            new ol.layer.Tile({
              source: new ol.source.OSM()
            })
              ,
            vectorLayer
          ],
          target: 'map',
          controls: ol.control.defaults({
            attributionOptions: /** @type {olx.control.AttributionOptions} */ ({
              collapsible: false
            })
          }),
          view: new ol.View({
            center: [1587927.09817072,5879650.90059265],
            zoom: 18
          })
        });

function addRoute(fromNumber, toNumber, routeType) {
    map.getLayers().pop();
    console.log("addRoute big"+ String(fromNumber));
    var baseUrl = 'http://localhost:8000/api/directions/';
    var geoJsonUrl = baseUrl + fromNumber + '&' + toNumber + '&' + routeType +'/?format=json';
    console.log("final url " + geoJsonUrl);
    map.getLayers().push(new ol.layer.Vector({
                source: new ol.source.GeoJSON({url: geoJsonUrl}),
                style:  new ol.style.Style({
                    stroke: new ol.style.Stroke({
                      color: 'purple',
                      width: 4
                    })
                  }),
                title: "Route",
                name: "Route"
            }));
    }
</code>

10. Now, go ahead and enter 1 and see the autocomplete in action; then, select Route To: and enter 2 to see the second floor options. Finally, click on GO! and see the magic happen:

<img src="./50790OS_11_08.jpg" width=400 height=400>

### How it works...

The internal workings of each step are shown, so let's go through all of them one at a time. We start off with the data import of our new room dataset as the basic starting and ending points of our new indoor routing tool. We lay out the top-level structure of our API with some new URLs, defining how we will call the new routing service, and then define these variables. Our regular expressions handle the correct data types that are passed in the URL without any exceptions.

These URL patterns are then used by api/views.py to actually accept the incoming room numbers and route types to generate our new route. This generation is split up into a few functions to increase usability. The get_room_centroid_node() function is necessary so that we can find the middle point of the room and then find the next nearest node on a network. We could also simply use the polygon geometry to find the nearest node but this can lead to ambiguity if the rooms are large and the entrances are close to each other. The centroid method is much more reliable and does not add too much overhead.

The run_route() function actually then runs find_closest_network_node(), which we created earlier, making things work well together. The run_route function then generates our GeoJSON result as it is passed in the start node ID, end node ID, and the route type.

The route_room_to_room() function is small as the heavy lifting has already been completed by our other functions. It simply inputs the URL parameters called by our API call, as seen in http://localhost:8000/api/directions/10010&20043&0. The final steps after step 6 are for the user interface. We need to provide the user with a list of rooms that are available to route from and to. The /api/rooms URL provides exactly this, delivering a JSON array of room numbers. The input fields are bootstrap inputs with Twitter Typeahead.js and Bloodhound.js to prefetch remote data. As a user, you simply enter a number and, bingo, a list appears. More detailed instructions on the JavaScript side of things are a little beyond the scope of this book, but these are, thankfully, kept to a minimum.

All in all, you now have a fully functional indoor mapping web application with a basic set of indoor 3D routing functions that you can expand on at any time.